In [1]:
"""Swiss Metro Price Changes by Age -- Base case -- senior

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',-0.215775,-20,20,0,'ASC_CAR')
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1,'ASC_SBB')
ASC_SM	 = Beta('ASC_SM',0.122635,-20,20,0,'ASC_SM')
B_CAR_COST	 = Beta('B_CAR_COST',-0.007514,-10,10,0,'B_CAR_COST')
B_HE	 = Beta('B_HE',-0.005887,-10,10,0,'B_HE')
B_SM_COST	 = Beta('B_SM_COST',-0.011591,-10,10,0,'B_SM_COST')
B_CAR_TIME	 = Beta('B_CAR_TIME',-0.013063,-10,10,0,'B_CAR_TIME')
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',-0.008814,-10,10,0,'B_TRAIN_TIME')
B_SM_TIME	 = Beta('B_SM_TIME',-0.010993,-10,10,0,'B_SM_TIME')
B_TRAIN_COST	 = Beta('B_TRAIN_COST',-0.029349,-10,10,0,'B_TRAIN_COST')
B_SENIOR	 = Beta('B_SENIOR',-1.864569,-10,10,0,'B_SENIOR')
B_GA	 = Beta('B_GA',0.518405,-10,80,0,'B_GA')
B_FIRST	 = Beta('B_FIRST',0.261506,-10,80,0,'B_FIRST')

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', 0.5 * SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Exclude
# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 1  ) + (  AGE == 2  ) + (  AGE == 3  ) + (  AGE == 4  ) + (  AGE == 6  ))>0
database.remove(exclude)

#No need for estimating the model (it is already estimated). We want to obtain the individual probabilities and the market shares
prob1 = models.logit(V,av,1)
prob2 = models.logit(V,av,2)
prob3 = models.logit(V,av,3)
#Instead of reporting the choice in the simulation file, the probability of the chosen can be printed

#Simulation output
simulate = {'choice': CHOICE,
            '01 Prob. Train': prob1,
            '02 Prob. SM': prob2,
            '03 Prob. Car': prob3,
            '04 Elast SM_COST Train': Derive(prob1,'SM_COST')*SM_COST/prob1,
            '05 Elast SM_COST SM': Derive(prob2,'SM_COST')*SM_COST/prob2,
            '06 Elast SM_COST Car': Derive(prob3,'SM_COST')*SM_COST/prob3}

biogeme  = bio.BIOGEME(database,simulate)
biogeme.modelName = "MNL_SM_price_base_senior"
results = biogeme.simulate()
print("Results=",results.describe())

07-04-2019:14:19:10,12 WARNING  [biogeme.py:143] The chosen alternative is not available for the following observations (rownumber[choice]): 243[3]-244[3]-245[3]-246[3]-247[3]-248[3]-249[3]-250[3]-251[3]-2187[3]-2188[3]-2189[3]-2190[3]-2191[3]-2192[3]-2193[3]-2194[3]-2195[3]-2223[3]-2224[3]-2225[3]-2226[3]-2227[3]-2228[3]-2229[3]-2230[3]-2231[3]-2817[3]-2818[3]-2819[3]-2820[3]-2821[3]-2822[3]-2823[3]-2824[3]-2825[3]-2826[3]-2827[3]-2828[3]-2829[3]-2830[3]-2831[3]-2832[3]-2833[3]-2834[3]-2862[3]-2863[3]-2864[3]-2865[3]-2866[3]-2867[3]-2868[3]-2869[3]-2870[3]-2970[3]-2971[3]-2972[3]-2973[3]-2974[3]-2975[3]-2976[3]-2977[3]-2978[3]-3042[3]-3043[3]-3044[3]-3045[3]-3046[3]-3047[3]-3048[3]-3049[3]-3050[3]-3303[3]-3304[3]-3305[3]-3306[3]-3307[3]-3308[3]-3309[3]-3310[3]-3311[3]-3321[3]-3322[3]-3323[3]-3324[3]-3325[3]-3326[3]-3327[3]-3328[3]-3329[3]-3348[3]-3349[3]-3350[3]-3351[3]-3352[3]-3353[3]-3354[3]-3355[3]-3356[3]-3357[3]-3358[3]-3359[3]-3360[3]-3361[3]-3362[3]-3363[3]-3364[3]-3365[3]-3429

Results=            choice  01 Prob. Train  02 Prob. SM  03 Prob. Car  \
count  432.000000      432.000000   432.000000    432.000000   
mean     1.601852        0.482496     0.437614      0.079891   
std      0.703848        0.246833     0.212416      0.093559   
min      1.000000        0.004425     0.139914      0.000000   
25%      1.000000        0.286369     0.270215      0.000000   
50%      1.000000        0.523925     0.390168      0.040795   
75%      2.000000        0.711910     0.572546      0.146117   
max      3.000000        0.858919     0.995575      0.480388   

       04 Elast SM_COST Train  05 Elast SM_COST SM  06 Elast SM_COST Car  
count              432.000000           432.000000            432.000000  
mean                 0.217683            -0.120709              0.168088  
std                  0.317262             0.115702              0.247712  
min                  0.000000            -0.552990              0.000000  
25%                  0.000000          